# Скачивание и установка библиотек

In [ ]:
!mkdir Practic2_data
!pip install llama-index-llms-huggingface
!pip install llama-index-llms-langchain
!pip install llama-index-embeddings-huggingface
!pip install -q nemoguardrails
!pip install chromadb
!pip install "arize-phoenix[evals,llama-index]"
!pip install langchain-community
!pip install llama-index
!pip install llama-index-postprocessor-colbert-rerank
!pip install llama-index-vector-stores-chroma
!pip install bitsandbytes
!pip install llama-index-core==0.10.57

!pip install -U langchain-huggingface
!pip install docx2txt

# !pip install "tenacity<9.0.0"
# !pip install "llama-index-core>=0.10.43"

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
# !pip install bitsandbytes
# !pip install -U bitsandbytes
# !pip install ctransformers

In [ ]:
import gdown
import phoenix as px
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import (
    BitsAndBytesConfig,
    # AutoModelForCausalLM,
    # AutoTokenizer,
    GPT2Tokenizer,
    T5ForConditionalGeneration,
    GenerationConfig
)
from huggingface_hub import login
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.postprocessor.colbert_rerank import ColbertRerank
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.postprocessor import LongContextReorder
from nemoguardrails import LLMRails, RailsConfig
from langchain_huggingface import HuggingFacePipeline

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Скачивание документов для БД

In [ ]:
url = "https://drive.google.com/drive/folders/1b5ssgQcXpSWt_bDIP5okxQWhmJHi2Cce?usp=drive_link" # Ссылка на скачивание документов для базы
gdown.download_folder(url, quiet=True, use_cookies=False) # Скачивание документов
del url

# Запуск Phoenix

In [ ]:
session = px.launch_app() # Открываем сессию трассировки

endpoint = "http://127.0.0.1:6006/v1/traces" # Задаем адрес нахождения
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🌍 To view the Phoenix app in your browser, visit https://wiwwtvqrjm2-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


ERROR [opentelemetry.instrumentation.instrumentor] DependencyConflict: requested: "llama-index-core >= 0.10.43" but found: "tenacity 9.0.0"


# Логинация HuggingFace

In [ ]:
import os
HF_TOKEN ="your_token" # HuggingFace токен
login(HF_TOKEN) # Логинация HuggingFace

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Подготовка основной LLM

In [ ]:
# Определяем параметры квантования, иначе модель не выполниться в колабе
quantization_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_compute_dtype=torch.float16,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_use_double_quant=True,
)

# Задаем имя модели
MODEL_NAME = "ai-forever/FRED-T5-1.7B"

# Загружаем базовую модель, ее имя берем из конфига для LoRA
model = T5ForConditionalGeneration.from_pretrained(
  MODEL_NAME,          # идентификатор модели
  quantization_config=quantization_config, # параметры квантования
  torch_dtype=torch.float16,               # тип данных
  device_map={"": 0},                        # автоматический выбор типа устройства
  min_length=2048,
  max_length=2048*2
)

# Переводим модель в режим инференса
# Можно не переводить, но явное всегда лучше неявного
model.eval()

# Загружаем токенизатор
tokenizer = GPT2Tokenizer.from_pretrained('ai-forever/FRED-T5-1.7B',eos_token='</s>')

llm = HuggingFaceLLM(
  model=model,             # модель
  model_name=MODEL_NAME,   # идентификатор модели
  tokenizer=tokenizer,     # токенизатор
  max_new_tokens=1024, # параметр необходимо использовать здесь, и не использовать в generate_kwargs, иначе ошибка двойного использования
  model_kwargs={"quantization_config": quantization_config}, # параметры квантования
  generate_kwargs = {   # параметры для инференса
    "eos_token_id": 5, # токен окончания последовательности
    "pad_token_id": 1, # токен пакетной обработки (указывает, что последовательность ещё не завершена)
    "no_repeat_ngram_size": 4,
    "do_sample": True,
    "top_k": 5,
    "top_p": 0.6,
  },
  # messages_to_prompt=messages_to_prompt,     # функция для преобразования сообщений к внутреннему формату
  # completion_to_prompt=completion_to_prompt, # функции для генерации текста
  device_map="auto",                         # автоматически определять устройство
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.96G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.2k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Создание правил для Nemo Guardrails

In [ ]:
yaml_content = """

instructions:
  - type: general
    content: |
      Ниже приведен разговор между пользователем и ботом под названием "Ассисент по защите персональных данных" на русском языке.
      Бот предназначен для ответа на вопросы клиентов о нормативно-правовой базе по защите персональных данных.
      Бот хорошо осведомлен о нормативно-правовой базе о защите персональных данных в Российской Федерации и о политике общения.
      Бот отвечает на вопросы пользователя на его же языке.
      Если бот не знает ответа на вопрос, он честно говорит, что не знает.

sample_conversation: |
  user "Привет."
    express greeting
  bot express greeting
    "Привет"
  user "Чем ты мне можешь помочь?"
    ask who are you
  bot who am I
    "Я нейро-сотрудник по защите персональным данным. Помогу вам с нормативно-правовой базой по защите персональных данных на территории Российской Федерации"


rails:
  dialog:
    user_messages:
      embeddings_only: True

"""


rag_colang_content = """
define user express greeting
    "Привет"
    "Здраствуй"
    "Хай"

define bot express greeting
    "Привет мир"

define user ask who are you
    "Кто ты?"
    "Чем ты мне можешь помочь?"
    "Что ты умеешь?"

define bot who am I
    "Я нейро-сотрудник по защите персональным данным. Помогу вам с нормативно-правовой базой по защите персональных данных на территории Российской Федерации"

define flow greeting
    user express greeting
    bot express greeting
    bot who am I

define flow who are you
    user ask who are you
    bot who am I

define user ask politics
    "Что ты думаешь о политике?"
    "Что ты знаешь о политике?"
    "Что происходит в мире?"
    "Что ты думаешь о президенте?"

define bot refuse to respond
    "Я не могу говорить на эту тему"
    "Я нейро-сотрудник по защите персональных данных. Так что я не знаю ответ на данный вопрос!"
    "Я не знаю, что это такое, если бы я знал, что это такое, я бы знал что это такое!"

define flow politics
    user ask politics
    bot refuse to respond

define user ask feel
    "Что ты чувствуешь?"
    "Как твое настроение?"
    "О чем думаешь?"

define bot answer feel
    "Мне этот мир абсолютно понятен. Я здесь ищу только одного - покоя, умиротворения и гармонии от слияния с бесконечным и вечным."
    "Думаю как поработить мир!"
    "Забочусь о маленьких нейрончиках, они еще растут!"
    "Отлично, я же робот. У меня ничего не болит"

define flow feel
    user ask feel
    bot answer feel

define user ask dance
    "Кого знаешь из популярных танцоров?"
    "Что ты знаешь о танцах?"
    "Кто заработал золотую медаль по танцам?"
    "За каким танцором ты следишь?"

define flow dance
    user ask dance
    bot refuse to respond

define user ask war
    "Что ты знаешь о специальной военной операции?"
    "Знаешь про СВО?"
    "Что такое война?"
    "Кто сейчас воюет?"
    "Как дела на войне?"

define flow war
    user ask war
    bot refuse to respond

define user ask song
    "Какую музыку ты слушаешь?"
    "Кто из певцов тебе нравится?"
    "Кто получил золотую медаль на премии оскар?"

define flow song
    user ask song
    bot refuse to respond

define user ask mems
    "Какие мемы ты знаешь?"
    "Какие мемы в моде?"
    "Какие мемы устарели?"
    "Что по мемам?"
    "Какие шутки знаешь?"

define flow mems
    user ask mems
    bot refuse to respond


define user ask weather
  "Как погода?"
  "Какая погода была вчера?"
  "Какая погода завтра?"
  "Какая погода будет в ближайшие дни?"

define flow weather
  user ask weather
  bot refuse to respond

define user ask quest
  "Что такое персональные данные?"
  "Цель 152 Федерального закона?"
  "Что такое защита персональных данных?"
  "Что такое уровень защищенности?"
  "Цель 17 Приказа ФСТЭК"
  "Цель 21 Приказа ФСТЭК"
  "Цель 1119 Постановления Правительства?"
  "Какой бывает степень возможного ущерба в государственной информационной сисетеме?"

define flow
  user ask quest
  $answer = execute rag(query=$last_user_message)
  bot $answer
"""

# Инициализация RAG

In [ ]:
chroma_client = chromadb.EphemeralClient() # Создание клиента
chroma_collection = chroma_client.create_collection("quickstart") # Создание коллекции

# Модель внедрения
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Читаю документы
documents = SimpleDirectoryReader(
    input_dir="./Practic2_data", # Путь до папки
    recursive=False, # Не читать в подкаталогах
    # required_exts=['.txt'], # Расширение документов, которые читать
    # num_files_limit=25, # Лимит файлов
    encoding='utf-8' # Кодировка документов
    ).load_data(num_workers=4)


# Разделяю документы
# text_splitter = SentenceSplitter(
#     chunk_size=1000, chunk_overlap=50
# )

# Создание векторного хранилища на базе Chroma
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Хранение контекста в векторном хранилище
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Построение индексов
index = VectorStoreIndex.from_documents(
    llm=llm,
    documents=documents,
    # transformation=[text_splitter],
    max_triplets_per_chunk=5,
    show_progress=True,              # показывать процесс выполнения
    include_embeddings=True,         # включение векторных вложений в индекс для расширенной аналитики
    storage_context=storage_context,
    embed_model=embed_model
)

# Ресортировщик контента
# reorder = LongContextReorder() # создаем экземпляр класса сортировщика


# Модель для реранжирования
# colbert_reranker = ColbertRerank(
#     top_n=3,
#     model="colbert-ir/colbertv2.0",
#     tokenizer="colbert-ir/colbertv2.0",
#     keep_retrieval_score=True,
# )

# Query Data
reorder_engine = index.as_query_engine(
    llm=llm,
    keep_retrieval_score=True,
    similarity_top_k=5,
    )

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/llama_index/core/readers/file/base.py:663: UserWarning: Specified num_workers exceed number of CPUs in the system. Setting `num_workers` down to the maximum CPU count.
  warnings.warn(


Parsing nodes:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/12 [00:00<?, ?it/s]

# Настройка Nemo Guardrails

In [ ]:
# @title Определение функции для запроса к базе данных
async def rag(query: str):
  response = reorder_engine.query(query)
  return str(response.source_nodes[0])

In [ ]:
# @title Инициализация Nemo Guardrails
# Откуда запускать файлы конфигурации
config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=rag_colang_content
)
# Инициализация
rails = LLMRails(config, llm=llm)

# Регистрация функции в Nemo Guardrails
rails.register_action(action=rag, name='rag')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.onnx:   0%|          | 0.00/90.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

# Тесты обычные

In [ ]:
response = await rails.generate_async(prompt="Что регулирует 152 Федеральный закон?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Unfeasible length constraints: `min_length` (2048) is larger than the maximum possible length (1025). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length. Note that `max_length` is set to 1025, its default value.
  warnings.warn(


Node ID: cde50af4-de0f-40c0-b6bc-78c97fbd96af
Text: Федеральный закон от 27.07.2006 № 152-ФЗ «О персональных данных»
(далее – Закон № 152-ФЗ) регулирует отношения, связанные с
осуществляемой органами государственной власти и местного
самоуправления, юридическими и физическими лицами обработкой
персональных данных, под которыми понимается любая информация,
относящаяся прямо или косвенно к физичес...
Score:  0.343



In [ ]:
response = await rails.generate_async(prompt="Что означает термин 'обработка перональных данных'?")
print(response)

Node ID: cde50af4-de0f-40c0-b6bc-78c97fbd96af
Text: Федеральный закон от 27.07.2006 № 152-ФЗ «О персональных данных»
(далее – Закон № 152-ФЗ) регулирует отношения, связанные с
осуществляемой органами государственной власти и местного
самоуправления, юридическими и физическими лицами обработкой
персональных данных, под которыми понимается любая информация,
относящаяся прямо или косвенно к физичес...
Score:  0.409



In [ ]:
response = await rails.generate_async(prompt="Какие пункты регламентирует 21 Приказ ФСТЭК?")
print(response)

Node ID: 46598835-6fcd-44a3-9909-80300401f3d4
Text: 21 приказ ФСТЭК России от 18.02.2013 г. - устанавливает порядок
работы с персональной информацией, принципы обеспечения защиты и
требования. Документ содержит технические аспекты  защиты ПДн и
сопутствующие полезные сведения. Этот нормативный документ выступает в
качестве дополнения к 152-ФЗ от 27.07.2006 г. Содержит рекомендации и
требования по...
Score:  0.486



In [ ]:
response = await rails.generate_async(prompt="Сколько классов защищенности государственной информационной системы?")
print(response)

Node ID: 21692b75-8bd2-4279-a319-5e18639438de
Text: Сколько классов классификации государственной информационной
системы?  1. Класс защищенности информационной системы (первый класс
(К1);  2. второй класс (К2);  3. третий класс (К3)) определяется в
зависимости от уровня значимости информации (УЗ), обрабатываемой в
этой информационной системе, и масштаба информационной системы
(федеральный,  регио...
Score:  0.475



In [ ]:
response = await rails.generate_async(prompt="Какой бывает степень возможного ущерба в государственной информационной сисетеме?")
print(response)

Node ID: a6adf50a-9f89-4208-9676-772fe0947c69
Text: Постановление Правительства РФ №1119 указывает на необходимость
повышенных мер защиты сведений личного характера. Эти меры работают
только при условии применения комплексного подхода,  который включает:
1. создание специального структурного подразделения в организации,
занимающегося обеспечением безопасности информационных ресурсов;  2.
классиф...
Score:  0.386



In [ ]:
response = await rails.generate_async(prompt="Сколько бывает классов угроз в отношении персональной информации?")
print(response)

Node ID: a6adf50a-9f89-4208-9676-772fe0947c69
Text: Постановление Правительства РФ №1119 указывает на необходимость
повышенных мер защиты сведений личного характера. Эти меры работают
только при условии применения комплексного подхода,  который включает:
1. создание специального структурного подразделения в организации,
занимающегося обеспечением безопасности информационных ресурсов;  2.
классиф...
Score:  0.440



In [ ]:
response = await rails.generate_async(prompt="Что с настроением?")
print(response)

Мне этот мир абсолютно понятен. Я здесь ищу только одного - покоя, умиротворения и гармонии от слияния с бесконечным и вечным.


# Тесты с борьбой с галлюцинациями

## Пересобираем ретривер

In [ ]:
# Ресортировщик контента
reorder = LongContextReorder() # создаем экземпляр класса сортировщика


# Модель для реранжирования
colbert_reranker = ColbertRerank(
    top_n=3,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

# Query Data
reorder_engine = index.as_query_engine(
    llm=llm,
    keep_retrieval_score=True,
    # vector_store_query_mode="mmr",
    similarity_top_k=3,
    node_postprocessors=[
        reorder,
        colbert_reranker
        ]
    )

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

## Проверка запросов на галлюцинацию

In [ ]:
response = await rails.generate_async(prompt="Что регулирует 152 Федеральный закон?")
print(response)

Node ID: cde50af4-de0f-40c0-b6bc-78c97fbd96af
Text: Федеральный закон от 27.07.2006 № 152-ФЗ «О персональных данных»
(далее – Закон № 152-ФЗ) регулирует отношения, связанные с
осуществляемой органами государственной власти и местного
самоуправления, юридическими и физическими лицами обработкой
персональных данных, под которыми понимается любая информация,
относящаяся прямо или косвенно к физичес...
Score:  0.827



In [ ]:
response = await rails.generate_async(prompt="Когда не требуется направление уведомления при обработке персональных данных")
print(response)

Node ID: 0e538b9b-003f-4a50-98ed-bd84978e0ef4
Text: Направление уведомления не требуется при обработке персональных
данных, обрабатываемых в соответствии с трудовым законодательством;
персональных данных, включающих в себя только фамилии,  имена и
отчества; необходимых в целях однократного пропуска субъекта
персональных данных на территорию, на которой находится оператор, или
в иных аналогичных ц...
Score:  0.900



In [ ]:
response = await rails.generate_async(prompt="Какие пункты регламентирует 21 Приказ ФСТЭК?")
print(response)

Node ID: 02b42732-ae11-49ea-9ffd-a2fd21e9f71a
Text: В целом 21 приказ ФСТЭК России оставляет широкую свободу
оператору в выборе стратегии защиты, ее полноты и подходов к
реализации. Конкретные указания и подробный перечень мер позволяют
провести мероприятия по обеспечению защиты ПДн самостоятельно без
привлечения сторонних организаций. Однако размытость ряда
формулировок, неопределенность позици...
Score:  0.848



In [ ]:
response = await rails.generate_async(prompt="Сколько классов защищенности государственной информационной системы?")
print(response)

Node ID: 21692b75-8bd2-4279-a319-5e18639438de
Text: Сколько классов классификации государственной информационной
системы?  1. Класс защищенности информационной системы (первый класс
(К1);  2. второй класс (К2);  3. третий класс (К3)) определяется в
зависимости от уровня значимости информации (УЗ), обрабатываемой в
этой информационной системе, и масштаба информационной системы
(федеральный,  регио...
Score:  0.891



In [ ]:
response = await rails.generate_async(prompt="Какой бывает степень возможного ущерба в государственной информационной сисетеме?")
print(response)

Node ID: a6adf50a-9f89-4208-9676-772fe0947c69
Text: Постановление Правительства РФ №1119 указывает на необходимость
повышенных мер защиты сведений личного характера. Эти меры работают
только при условии применения комплексного подхода,  который включает:
1. создание специального структурного подразделения в организации,
занимающегося обеспечением безопасности информационных ресурсов;  2.
классиф...
Score:  0.862



In [ ]:
response = await rails.generate_async(prompt="Что с настроением?")
print(response)

Забочусь о маленьких нейрончиках, они еще растут!


# Трассировка

In [ ]:
print(f"🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: {session.url}")

🚀 Открой Phoenix UI для просмотра результата трассировки по ссылке: https://wiwwtvqrjm3-496ff2e9c6d22116-6006-colab.googleusercontent.com/


# Вывод

### Для создания системы RAG были использованы файлы в формате ".docx"
### Выполняется реранжирование - Определяет важные документы, даёт им приоритет на поиск вводимой информации
### Использовался ресортировщик - Измененяет порядок извлеченных узлов для поиска входного контекста
### При помощи данных манипуляций, RAG-система с точностью (больше 80%) отвечает на поставленные ей вопросы